In [ ]:
# 크롤링 : 데이터 수집

from selenium import webdriver
from bs4 import BeautifulSoup as soups

def search_selenium(search_name, search_path, search_limit) :
    search_url = "https://www.google.com/search?q=" + str(search_name) + "&hl=ko&tbm=isch"
    
    browser = webdriver.Chrome(r'C:/Users/user/Documents/chromedriver/chromedriver.exe')
    browser.get(search_url)
    
    image_count = len(browser.find_elements_by_tag_name("img"))
    
    print("로드된 이미지 개수 : ", image_count)

    browser.implicitly_wait(2)

    for i in range( search_limit ) :
        image = browser.find_elements_by_tag_name("img")[i]
        image.screenshot("C:/Users/user/Pictures/AI Referee" + str(i) + ".png")

    browser.close()

if __name__ == "__main__" :

    search_name = input("검색하고 싶은 키워드 : ")
    search_limit = int(input("원하는 이미지 수집 개수 : "))
    search_path = "Your Path"
    search_selenium(search_name, search_path, search_limit)


In [ ]:
# 모델 설계

import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

# 폴더 생성

FoulOrNot = "/tmp/FoulOrNot"

try:
    os.mkdir('/tmp/RefereeAI')
    os.mkdir('/tmp/RefereeAI/training')
    os.mkdir('/tmp/RefereeAI/testing')
    os.mkdir('/tmp/RefereeAI/training/foul')
    os.mkdir('/tmp/RefereeAI/training/nofoul')
    os.mkdir('/tmp/RefereeAI/testing/foul')
    os.mkdir('/tmp/RefereeAI/testing/nofoul')
except OSError:
    pass


# 데이터 스플릿 : FoulOrNot에 있는 데이터를 훈련용, 테스트용 데이터로 스플릿

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length] 
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

FOUL_SOURCE_DIR = "/tmp/FoulOrNot/Foul/"
TRAINING_FOUL_DIR = "/tmp/RefereeAI/training/foul/"
TESTING_FOUL_DIR = "/tmp/RefereeAI/testing/foul/"
NOFOUL_SOURCE_DIR = "/tmp/FoulOrNot/NoFoul/"
TRAINING_NOFOUL_DIR = "/tmp/RefereeAI/training/nofoul/"
TESTING_NOFOUL_DIR = "/tmp/RefereeAI/testing/nofoul/"

split_size = .8
split_data(FOUL_SOURCE_DIR, TRAINING_FOUL_DIR, TESTING_FOUL_DIR, split_size)
split_data(NOFOUL_SOURCE_DIR, TRAINING_NOFOUL_DIR, TESTING_NOFOUL_DIR, split_size)


# 데이터양 파악
print(len(os.listdir('/tmp/RefereeAI/training/foul/')))
print(len(os.listdir('/tmp/RefereeAI/training/nofoul/')))
print(len(os.listdir('/tmp/RefereeAI/testing/foul/')))
print(len(os.listdir('/tmp/RefereeAI/testing/nofoul/')))


# 컨볼루션 신경망 구축
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

TRAINING_DIR = "/tmp/RefereeAI/training/"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=10,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/RefereeAI/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=10,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

# 모델 학습
history = model.fit(train_generator,
                              epochs=50,
                              verbose=1,
                              validation_data=validation_generator)

In [ ]:
# 테스트기 : 사진 업로드 하면 파울인지 아닌지 판단


import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is not foul")
  else:
    print(fn + " is foul")